In [1]:
#coding:utf-8
import torch as t
import numpy as np
from torch import nn
import ipdb

In [2]:
import sys
sys.path.append('D:\\Jupyter\\Python\\ATAE-LSTM')
import Ipynb_importer
from config import opt
from data.Embedding import emb
from models.BasicModule import BasicModule
from data.AspClas_ import AspClas

Embedding : have input words : 1
Embedding : have input words : 2
Embedding : have input words : 4
Embedding : have input words : 8
Embedding : have input words : 16
Embedding : have input words : 32
Embedding : have input words : 64
Embedding : have input words : 128
Embedding : have input words : 256
Embedding : have input words : 512
Embedding : have input words : 1024
Embedding : have input words : 2048
Embedding : have input words : 4096
Embedding : have input words : 8192
Embedding : have input words : 16384


In [3]:
sys.path.append('D:\\Jupyter\\Python')
from MyGRU.MyGRU import MyGRU

importing Jupyter notebook from D:\Jupyter\Python\MyGRU\MyGRU.ipynb
Automatic pdb calling has been turned ON


In [4]:
class AE_MyGRU(BasicModule):
    def __init__(self):
        super(AE_MyGRU, self).__init__()
        
        self.embedding = emb._make_layer_()
        
        self.gru = MyGRU(opt.hidden_size*2, opt.hidden_size)
        self.h0 = t.randn(1, opt.hidden_size)
        
        self.lin = nn.Linear(opt.hidden_size, opt.classes)
        self.tanh = t.nn.Tanh()
        self.softmax = nn.Softmax(dim=0)
        
        return
    
    def forward(self, x):
        # word representation
        w = x[0]
        N = len(w)
        # aspect term
        v = x[1]
        # assert len(v)==1  # use mean()
        
        # e.g.
        # w torch.Size([16])
        # v torch.Size([1])
        
        e1 = self.embedding(x[0])
        e2 = self.embedding(x[1]).mean(dim=0).view(-1).expand(e1.size())
        # e.g.
        # e1 torch.Size([16, 300])
        # e2 torch.Size([1, 300]) -> torch.Size([16, 300])
        
        wv = t.cat((e1.view(N,1,opt.hidden_size), e2.view(N,1,opt.hidden_size)), dim=-1)
        # e.g.
        # wv torch.Size([16, 1, 600])
        
        out, hn = self.gru([wv, self.h0])
        # e.g.
        # out torch.Size([16, 1, 300])
        # hn torch.Size([1, 1, 300])
        
        y = self.softmax(self.lin(hn.view(opt.hidden_size)))
        # e.g.
        # y torch.Size([3])
        
        return y

In [5]:
if __name__=='__main__':
    %pdb on
    testDataset = AspClas(opt.base_root+'/data/restaurants-trial.xml')
    model = AE_MyGRU()
    for text, aspect, sentiment in testDataset:
        x = (text, aspect)
        y = model(x)
        print(y)
    print(len(list(model.parameters())))

Automatic pdb calling has been turned ON
tensor([0.2178, 0.2391, 0.5430], grad_fn=<SoftmaxBackward>)
tensor([0.2510, 0.5917, 0.1573], grad_fn=<SoftmaxBackward>)
tensor([0.3761, 0.2859, 0.3380], grad_fn=<SoftmaxBackward>)
tensor([0.3287, 0.1883, 0.4830], grad_fn=<SoftmaxBackward>)
tensor([0.2324, 0.4354, 0.3323], grad_fn=<SoftmaxBackward>)
tensor([0.1823, 0.4187, 0.3990], grad_fn=<SoftmaxBackward>)
tensor([0.2554, 0.3386, 0.4059], grad_fn=<SoftmaxBackward>)
tensor([0.2344, 0.3226, 0.4431], grad_fn=<SoftmaxBackward>)
tensor([0.1437, 0.4123, 0.4440], grad_fn=<SoftmaxBackward>)
tensor([0.2345, 0.2687, 0.4968], grad_fn=<SoftmaxBackward>)
tensor([0.2151, 0.1001, 0.6849], grad_fn=<SoftmaxBackward>)
tensor([0.2350, 0.5388, 0.2261], grad_fn=<SoftmaxBackward>)
tensor([0.5436, 0.2525, 0.2040], grad_fn=<SoftmaxBackward>)
tensor([0.2174, 0.4648, 0.3178], grad_fn=<SoftmaxBackward>)
tensor([0.2675, 0.2823, 0.4502], grad_fn=<SoftmaxBackward>)
tensor([0.1817, 0.2311, 0.5872], grad_fn=<SoftmaxBackward>)